# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd)'

# Read 100 questions list

In [3]:
file = 'Excel/' + os.listdir('Excel')[-1]
print(file)
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan
df.head()

Excel/Source3_3.xlsx


,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/,https://www.autism.org/autism-prognosis/,https://autismcanada.org/autism-explained/,https://autismcanada.org/autism-explained/cooccurring-conditions/,https://autismcanada.org/autism-explained/screening-tools/,https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd)
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN,NaN,https://autismcanada.org/autism-explained/diag...,NaN,NaN,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN,NaN,NaN,NaN,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN,Autism is a neurodevelopmental disorder that b...,"Autism Spectrum Disorder, or autism, is a neur...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
114,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [8]:
x = soup.find('div',{'id':'sf-accordion'}).text
print(x)






                                What are autism spectrum disorders?
                            



                                Autism spectrum disorders (ASD) are a group of complex brain development disorders. This umbrella term covers conditions such as autism and Asperger syndrome. These disorders are characterized by difficulties in social interaction and communication and a restricted and repetitive repertoire of interests and activities. 








                                How common are autism spectrum disorders?
                            



                                Reviews estimate that 1 child in 160 has an autism spectrum disorder. This estimate represents an average figure, and reported prevalence varies substantially across studies. Some recent studies have, however, reported rates that are substantially higher. 








                                Do people with an autism spectrum disorder always suffer from intellectual disability?
           

In [23]:
l1=[]
l2=[]
for i in x.split("\n\n\n\n"):
    p=i.strip() 
    if(p!=''):
        print(p)
        print('*'*100)
        if('?' in p):
            l1.append(p)
        else:
            l2.append(p)

What are autism spectrum disorders?
****************************************************************************************************
Autism spectrum disorders (ASD) are a group of complex brain development disorders. This umbrella term covers conditions such as autism and Asperger syndrome. These disorders are characterized by difficulties in social interaction and communication and a restricted and repetitive repertoire of interests and activities.
****************************************************************************************************
How common are autism spectrum disorders?
****************************************************************************************************
Reviews estimate that 1 child in 160 has an autism spectrum disorder. This estimate represents an average figure, and reported prevalence varies substantially across studies. Some recent studies have, however, reported rates that are substantially higher.
******************************************

In [25]:
len(l1),len(l2)

(7, 7)

# Check which questions are similar

In [19]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [20]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [21]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [22]:
sim(l1)

Website -->  What are autism spectrum disorders? (Index 0)
100 questions list -->  What are the Autism Spectrum Disorders (ASD)? (Index 0)
0.75
****************************************************************************************************
Website -->  How common are autism spectrum disorders? (Index 1)
100 questions list -->  What are the Autism Spectrum Disorders (ASD)? (Index 0)
0.6
****************************************************************************************************
Website -->  Do people with an autism spectrum disorder always suffer from intellectual disability? (Index 2)
100 questions list -->  What are the Autism Spectrum Disorders (ASD)? (Index 0)
0.2
****************************************************************************************************
Website -->  How early can an autism spectrum disorder be recognized in children? (Index 3)
100 questions list -->  What is the prognosis of children with Autism? (Index 26)
0.2857142857142857
*****************

In [ ]:
# Website, 100Q
# 0, 0


In [32]:
l2[5]

'Scientific evidence suggests that various factors, both genetic and environmental, contribute to the onset of autism spectrum disorders by influencing early brain development.'

In [31]:
for i,j in enumerate(df['Question']):
    if('causes' in j):
        print(i,j)

13 What are some causes of behavioral change in autistic children?


In [33]:
# Directly add answer to existing dataframe
l4 = [[0, 0]]

for i in l4:
    df[link].loc[i[1]] = l2[i[0]]

C:\Users\aparg\anaconda3\envs\ASD_scraping\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [34]:
l1

['What are autism spectrum disorders?',
 'How common are autism spectrum disorders?',
 'Do people with an autism spectrum disorder always suffer from intellectual disability?',
 'How early can an autism spectrum disorder be recognized in children?',
 'What can parents do to help their child with an autism spectrum disorder?',
 'What causes autism spectrum disorders?',
 'Are childhood vaccines responsible for autism spectrum disorders?']

In [35]:
# update questions list
f = [i[0] for i in l4]
f = set(range(len(l1)))-set(f)
l1=[l1[i] for i in f]
l1

['How common are autism spectrum disorders?',
 'Do people with an autism spectrum disorder always suffer from intellectual disability?',
 'How early can an autism spectrum disorder be recognized in children?',
 'What can parents do to help their child with an autism spectrum disorder?',
 'What causes autism spectrum disorders?',
 'Are childhood vaccines responsible for autism spectrum disorders?']

In [36]:
# select answers to questions in list 'l'
l2 = [l2[i] for i in f]

In [37]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd)
0,How common are autism spectrum disorders?,Reviews estimate that 1 child in 160 has an au...
1,Do people with an autism spectrum disorder alw...,The level of intellectual functioning is extre...
2,How early can an autism spectrum disorder be r...,Identifying an autism spectrum disorder is dif...
3,What can parents do to help their child with a...,Parents have an essential role in providing su...
4,What causes autism spectrum disorders?,Scientific evidence suggests that various fact...
5,Are childhood vaccines responsible for autism ...,Available epidemiological data show that there...


In [38]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3.head(10)

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/,https://www.autism.org/autism-prognosis/,https://autismcanada.org/autism-explained/,https://autismcanada.org/autism-explained/cooccurring-conditions/,https://autismcanada.org/autism-explained/screening-tools/,https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd)
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Autism spectrum disorders (ASD) are a group of...
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN,NaN,https://autismcanada.org/autism-explained/diag...,NaN,NaN,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN,NaN,NaN,NaN,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN,Autism is a neurodevelopmental disorder that b...,"Autism Spectrum Disorder, or autism, is a neur...",NaN,NaN,NaN
5,What are Screening Tools for Autism?,NaN,Early intervention can make a big difference f...,NaN,NaN,NaN,NaN,NaN,\nDevelopmental Screening ToolsScreening tools...,NaN
6,How Common Is It?,NaN,"For many years a diagnosis of autism was rare,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,What are the Signs of Autism?,NaN,Many signs can indicate that an individual may...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,What should I do next?,NaN,When preparing to talk to your medical provide...,"Attend one or more parent support groups, o...",NaN,NaN,NaN,NaN,NaN,NaN
9,How can you tell Autism from Asperger’s Syndrome?,NaN,NaN,Asperger syndrome was phased out with the int...,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df3.iloc[:,[0,-1]].dropna()

,Question,https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd)
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorders (ASD) are a group of...
32,Are childhood vaccines responsible for autism ...,Available epidemiological data show that there...
33,How early can an autism spectrum disorder be r...,Identifying an autism spectrum disorder is dif...
34,What can parents do to help their child with a...,Parents have an essential role in providing su...
35,How common are autism spectrum disorders?,Reviews estimate that 1 child in 160 has an au...
36,What causes autism spectrum disorders?,Scientific evidence suggests that various fact...
37,Do people with an autism spectrum disorder alw...,The level of intellectual functioning is extre...


In [45]:
df3.shape

(125, 10)

# Save Output

In [46]:
df3.to_excel('Excel/Source4.xlsx',index=False)